In [1]:
# step1:read data in
#read data in
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score # use to calculate accuracy
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pd.set_option('display.max_columns', None)


#https://www.kaggle.com/lucidlenn/data-analysis-and-classification-using-xgboost

# sep: to decide the delimiter, default is comma in csv
# header: to decide whether the data frame has header or not, if don't want header,set header=None
# names = ['var1','var2',...] to list column names when header=None
# index_col is assign which column as index, 0 means to assign the first column as index, 1 means assign the second column to index
# skiprows: to skip rows when read in csv file, example skiprows=[0,3,5,10]
# skipfooter: number of lines at bottom of file to skip
# nrows: number of rows to read in 
# na_values: additional strings to recognize as NA, example na_values= ['NULL','NAN']
# parse_dates = assign a column list you want to parse dates on
# low_memory = False to ensure no mixed types of data. Pandas will guess the dtypes of each column, that is very memroy consuming and pandas need to read all the dataset from begin to the end to decide dtypes of each columns

loan = pd.read_csv("P:\Kaggle Project\loan\loan.csv",low_memory=False)
loan.info()
loan.head()

C:\Users\gqbwang\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 74 columns):
id                             887379 non-null int64
member_id                      887379 non-null int64
loan_amnt                      887379 non-null float64
funded_amnt                    887379 non-null float64
funded_amnt_inv                887379 non-null float64
term                           887379 non-null object
int_rate                       887379 non-null float64
installment                    887379 non-null float64
grade                          887379 non-null object
sub_grade                      887379 non-null object
emp_title                      835917 non-null object
emp_length                     842554 non-null object
home_ownership                 887379 non-null object
annual_inc                     887375 non-null float64
verification_status            887379 non-null object
issue_d                        887379 non-null object
loan_status          

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-2015,171.62,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,NaN,Sep-2013,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,2.0,0.0,2956.0,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-2014,649.91,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,35.0,NaN,10.0,0.0,5598.0,21.0,37.0,f,0.0,0.0,12226.302212,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-2015,357.48,NaN,Jan-2015,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-2016,67.79,Feb-2016,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# 2. target variable
loan['loan_status'].value_counts()

def loandec(var):
    if var in ['Charged Off','Late (31-120 days)','Late (16-30 days)','Default']:
        return 'Late/default/chargedoff'
    elif var in ['Current']:
        return 'Current'
    else:
        return 'Other'
    
loan['loan_status_label'] = loan['loan_status'].apply(loandec)
loan['loan_status_label'].value_counts()

Current                    601779
Other                      225185
Late/default/chargedoff     60415
Name: loan_status_label, dtype: int64

In [8]:
loan_no_others = loan.loc[loan['loan_status_label'] != 'Other',:]
loan_no_others['loan_status_label'].value_counts()

Current                    601779
Late/default/chargedoff     60415
Name: loan_status_label, dtype: int64

In [10]:
def cha(var):
    if var == 'Late/default/chargedoff':
        return 1
    else:
        return 0
loan_no_others['Target'] = loan_no_others['loan_status_label'].apply(cha)
loan_no_others['Target'].value_counts()

# drop two variable loan_status, loan_status_label
loan_target = loan_no_others.drop(columns=['loan_status', 'loan_status_label'])
loan_target.head()

C:\Users\gqbwang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,Target
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.00,0.00,1008.71,1008.71,456.46,435.17,0.0,117.08,1.11,Apr-2013,119.66,NaN,Sep-2013,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,f,766.90,766.90,3242.17,3242.17,2233.10,1009.07,0.0,0.00,0.00,Jan-2016,67.79,Feb-2016,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,1069639,1304742,7000.0,7000.0,7000.0,60 months,15.96,170.08,C,C5,Southern Star Photography,8 years,RENT,47004.0,Not Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/18/11 > I am planning o...,debt_consolidation,Loan,280xx,NC,23.51,0.0,Jul-2005,1.0,NaN,NaN,7.0,0.0,17726.0,85.6,11.0,f,1889.15,1889.15,8136.84,8136.84,5110.85,3025.99,0.0,0.00,0.00,Jan-2016,170.08,Feb-2016,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,1071795,1306957,5600.0,5600.0,5600.0,60 months,21.28,152.39,F,F2,NaN,4 years,OWN,40000.0,Source Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I own a small h...,small_business,Expand Business & Buy Debt Portfolio,958xx,CA,5.55,0.0,Apr-2004,2.0,NaN,NaN,11.0,0.0,5210.0,32.6,13.0,f,0.00,0.00,646.02,646.02,162.02,294.94,0.0,189.06,2.09,Apr-2012,152.39,NaN,Aug-2012,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,1071570,1306721,5375.0,5375.0,5350.0,60 months,12.69,121.45,B,B5,Starbucks,< 1 year,RENT,15000.0,Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/16/11 > I'm trying to b...,other,Building my credit history.,774xx,TX,18.08,0.0,Sep-2004,0.0,NaN,NaN,2.0,0.0,9279.0,36.5,3.0,f,0.00,0.00,1476.19,1469.34,673.48,533.42,0.0,269.29,2.52,Nov-2012,121.45,NaN,Mar-2013,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [15]:
# change issue_d to datetime
loan_target['issue_d_date'] = pd.to_datetime(loan_target['issue_d'],format='%b-%Y')
loan_target['last_pymnt_d'] = pd.to_datetime(loan_target['last_pymnt_d'],format='%b-%Y')


loan_target['days_from_issue_to_lastPayment'] = (loan_target['last_pymnt_d'] - loan_target['issue_d_date']).dt.days
loan_target.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,Target,issue_d_date,days_from_issue_to_lastPayment
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.0,0.0,1008.71,1008.71,456.46,435.17,0.0,117.08,1.11,2013-04-01,119.66,NaN,Sep-2013,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2011-12-01,487.0
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,f,766.9,766.9,3242.17,3242.17,2233.10,1009.07,0.0,0.00,0.00,2016-01-01,67.79,Feb-2016,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2011-12-01,1492.0


In [17]:
# drop some variables may not need in entire analyssi
loan_drop = loan_target.drop(columns=['id','member_id','emp_title','issue_d','url','desc','earliest_cr_line','last_pymnt_d','last_credit_pull_d',
                                    'issue_d_date','next_pymnt_d'])
loan_drop.head(2)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,Target,days_from_issue_to_lastPayment
1,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,n,car,bike,309xx,GA,1.00,0.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.0,0.0,1008.71,1008.71,456.46,435.17,0.0,117.08,1.11,119.66,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,487.0
4,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,1 year,RENT,80000.0,Source Verified,n,other,Personal,972xx,OR,17.94,0.0,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,f,766.9,766.9,3242.17,3242.17,2233.10,1009.07,0.0,0.00,0.00,67.79,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1492.0


In [24]:
# check missing values see any extra variable need to drop
missing_col_percentage = pd.DataFrame((loan_drop.isnull().sum()/len(loan_drop)).sort_values(ascending=False))
missing_col_percentage.columns=['MissingPercent']
missing_col_percentage.head(10)

,MissingPercent
dti_joint,0.999334
verification_status_joint,0.999331
annual_inc_joint,0.999331
il_util,0.982659
mths_since_rcnt_il,0.980679
total_cu_tl,0.980151
inq_fi,0.980151
all_util,0.980151
max_bal_bc,0.980151
open_rv_24m,0.980151


In [26]:
# for variables has missing values >= 0.95 add indicators
col_remove = missing_col_percentage.loc[missing_col_percentage['MissingPercent'] >= 0.95,:]
col_missing_lnd = col_remove.index.tolist()
print(col_missing_lnd)

# isna() will change to bool and *1 will change to 0 and 1
for x in col_missing_lnd:
    loan_drop[x+'lnd'] = loan_drop[x].isna()*1


loan_addlnd= loan_drop.drop(columns=col_missing_lnd)
loan_addlnd.head()

['dti_joint', 'verification_status_joint', 'annual_inc_joint', 'il_util', 'mths_since_rcnt_il', 'total_cu_tl', 'inq_fi', 'all_util', 'max_bal_bc', 'open_rv_24m', 'inq_last_12m', 'total_bal_il', 'open_il_24m', 'open_il_12m', 'open_il_6m', 'open_acc_6m', 'open_rv_12m']


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,Target,days_from_issue_to_lastPayment,dti_jointlnd,verification_status_jointlnd,annual_inc_jointlnd,il_utillnd,mths_since_rcnt_illnd,total_cu_tllnd,inq_filnd,all_utillnd,max_bal_bclnd,open_rv_24mlnd,inq_last_12mlnd,total_bal_illnd,open_il_24mlnd,open_il_12mlnd,open_il_6mlnd,open_acc_6mlnd,open_rv_12mlnd
1,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,n,car,bike,309xx,GA,1.00,0.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.00,0.00,1008.71,1008.71,456.46,435.17,0.0,117.08,1.11,119.66,0.0,NaN,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN,1,487.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,1 year,RENT,80000.0,Source Verified,n,other,Personal,972xx,OR,17.94,0.0,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,f,766.90,766.90,3242.17,3242.17,2233.10,1009.07,0.0,0.00,0.00,67.79,0.0,NaN,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN,0,1492.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,7000.0,7000.0,7000.0,60 months,15.96,170.08,C,C5,8 years,RENT,47004.0,Not Verified,n,debt_consolidation,Loan,280xx,NC,23.51,0.0,1.0,NaN,NaN,7.0,0.0,17726.0,85.6,11.0,f,1889.15,1889.15,8136.84,8136.84,5110.85,3025.99,0.0,0.00,0.00,170.08,0.0,NaN,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN,0,1492.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
8,5600.0,5600.0,5600.0,60 months,21.28,152.39,F,F2,4 years,OWN,40000.0,Source Verified,n,small_business,Expand Business & Buy Debt Portfolio,958xx,CA,5.55,0.0,2.0,NaN,NaN,11.0,0.0,5210.0,32.6,13.0,f,0.00,0.00,646.02,646.02,162.02,294.94,0.0,189.06,2.09,152.39,0.0,NaN,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN,1,122.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9,5375.0,5375.0,5350.0,60 months,12.69,121.45,B,B5,< 1 year,RENT,15000.0,Verified,n,other,Building my credit history.,774xx,TX,18.08,0.0,0.0,NaN,NaN,2.0,0.0,9279.0,36.5,3.0,f,0.00,0.00,1476.19,1469.34,673.48,533.42,0.0,269.29,2.52,121.45,0.0,NaN,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN,1,336.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [37]:
missing_col_percentage2 = pd.DataFrame((loan_addlnd.isnull().sum()/len(loan_addlnd)).sort_values(ascending=False))
missing_col_percentage2.columns=['MissingPercent']
missing_col = missing_col_percentage2.loc[missing_col_percentage2['MissingPercent']>0,:]

dtype = pd.DataFrame(loan_addlnd.dtypes)
dtype.columns=['dtypes']
dtype

missing_dtype = pd.merge(missing_col,dtype,left_index=True,right_index=True,how='inner')
missing_dtype.sort_values(by='dtypes')

,MissingPercent,dtypes
mths_since_last_record,0.837480,float64
mths_since_last_major_derog,0.730513,float64
mths_since_last_delinq,0.499246,float64
tot_coll_amt,0.020798,float64
total_rev_hi_lim,0.020798,float64
tot_cur_bal,0.020798,float64
days_from_issue_to_lastPayment,0.013626,float64
revol_util,0.000461,float64
collections_12_mths_ex_med,0.000009,float64
emp_length,0.055195,object


In [49]:
loan_addlnd.groupby(loan_addlnd['emp_length'].map(str)).agg({'Target':np.mean},dropna=False)


2500.0